In [63]:
# Exo 1 - Préparation des données 

# importation des données
import pandas as pd 
import numpy as np 
# import du fichier
data = pd.read_csv("synthetic.csv")

# Visualisation des données
print(data.head())


      Attr_A     Attr_B       Attr_C      Attr_D      Attr_E       Attr_F  \
0  12.478064  14.784992  1247.541877  100.962061   52.462177  1089.398211   
1   8.264345   8.854181  1389.686814   99.526529   65.106526  1186.523399   
2   9.875571  13.292442   779.077401  123.993772  104.699796  1201.722480   
3   9.207661   9.346913   965.468523   89.176009  102.628284   743.913507   
4   8.863842  12.542969  1096.386230  106.595385  131.813380   883.059615   

      Attr_G     Attr_H     Attr_I      Attr_J       Attr_K     Attr_L  \
0  10.575834   8.375407  10.288159  110.746551   994.367610   9.069350   
1   9.500485  10.088058   9.371983   78.210274   943.089589   9.988919   
2   9.545266  14.266238   9.703551   86.252483  1082.989190  10.084217   
3   9.777953  11.613946   8.912059   96.727873   812.800511   8.621781   
4  10.092974  13.556029  11.649982   21.566576   971.083175  10.072271   

        Attr_M      Attr_N  Class  
0  1027.953917  109.672758      1  
1  1120.317724   83.

In [64]:
from sklearn.model_selection import train_test_split


def neural_split_data(data, target_attribute):

    # Séparez les données et les labels afin de pouvoir séparer en jeu d'entrainement
    # de test et de validation
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[target_attribute]),
                                                        data[target_attribute], test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15/0.85,
                                                    random_state=42)
    return X_train, y_train, X_val, y_val, X_test, y_test

def one_hot_encode(y, num_classes):
    """Converts a vector of class indices into a one-hot encoded matrix."""
    y_array = np.array(y)  # Convert the pandas Series to a NumPy array if necessary
    return np.eye(num_classes)[y_array.reshape(-1)]


#print (X_train)

In [65]:
# Calcul de l'entropie

"""
L'entropie est une mesure de l'incertitude associée à une variable aléatoire.
"""

def entropie(dataframe , attribut_cible):  
    # Calcul de la probabilité de chaque classe
    compte_classe = dataframe[attribut_cible].value_counts()
    #print(compte_classe)
    proba = compte_classe / compte_classe.sum()
    #print(proba) 
    # Calcul de l'entropie
    entropie = - (proba * np.log2(proba+ np.finfo(float).eps)).sum() # éviter log2(0)
    return entropie

# Test de la fonction
print(entropie(data, 'Attr_A'))

11.166163082645376


In [66]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# Cross Entropy function.
def cross_entropy(y_pred, y_true):
 
    # computing softmax values for predicted values
    y_pred = softmax(y_pred)
    loss = 0
     
    # Doing cross entropy Loss
    for i in range(len(y_pred)):
 
        # Here, the loss is computed using the
        # above mathematical formulation.
        loss = loss + (-1 * y_true[i]*np.log(y_pred[i]))
 
    return loss
 
# y_true: True Probability Distribution
y_true = [1, 0, 0, 0, 0]
 
# y_pred: Predicted values for each calss
y_pred = [10, 5, 3, 1, 4]

# Calling the cross_entropy function by passing
# the suitable values
cross_entropy_loss = cross_entropy(y_pred, y_true)
 
print("Cross Entropy Loss: ", cross_entropy_loss)


print("Debugging Entropy Calculation:")
debug_data = pd.DataFrame({"Class": [1, 1, 1, 0, 0, 2, 2]})
print(f"Calculated Entropy: {entropie(debug_data, 'Class')}")

Cross Entropy Loss:  0.010199795719758053
Debugging Entropy Calculation:
Calculated Entropy: 1.556656707462822


In [67]:
import numpy as np

class NeuralNetwork:
    def __init__(self, architecture, activation='relu'):
        self.architecture = architecture
        self.activation = activation
        self.weights = []
        self.biases = []
        self.initialize_parameters()

    def initialize_parameters(self):
        np.random.seed(42)
        for i in range(1, len(self.architecture)):
            if self.activation == 'relu':
                # He initialization for ReLU
                weight_matrix = np.random.randn(self.architecture[i], self.architecture[i - 1]) * np.sqrt(2. / self.architecture[i - 1])
            else:
                # Xavier initialization for other activations
                weight_matrix = np.random.randn(self.architecture[i], self.architecture[i - 1]) * np.sqrt(1. / self.architecture[i - 1])
            bias_vector = np.zeros((self.architecture[i], 1))
            self.weights.append(weight_matrix)
            self.biases.append(bias_vector)

    def activation_function(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError("Unsupported activation function. Choose 'relu' or 'tanh'.")

    def activation_derivative(self, x):
        if self.activation == 'relu':
            return (x > 0).astype(float)
        elif self.activation == 'tanh':
            return 1 - np.tanh(x)**2
        else:
            raise ValueError("Unsupported activation function for derivative. Choose 'relu' or 'tanh'.")

    def forward_propagation(self, X):
        activations = [X.T]
        weighted_inputs = []
        for i in range(len(self.weights)):
            if i == 0:
                activations[-1]= activations[-1].T
            print(f"Layer {i}:")
            print(f"Activations shape: {activations[-1].shape}")
            print(f"Weights shape: {self.weights[i].shape}")
            weighted_input = np.dot(self.weights[i],activations[-1] ) + self.biases[i]
            weighted_inputs.append(weighted_input)
            activations.append(self.activation_function(weighted_input))
        return activations, weighted_inputs
    
    def backward_propagation(self, X, y, activations, weighted_inputs):
        m = X.shape[1]
        delta = activations[-1] - y
        dw = []
        db = []
        
        for i in reversed(range(len(self.weights))):
            dw_i = np.dot(delta, activations[i].T) / m
            db_i = np.sum(delta, axis=1, keepdims=True) / m
            dw.insert(0, dw_i)
            db.insert(0, db_i)
            
            if i > 0:
                delta = np.dot(self.weights[i].T, delta) * self.activation_derivative(weighted_inputs[i - 1])

        return {'dw': dw, 'db': db}

    # def backward_propagation(self, X, y, activations, weighted_inputs):
    #     m = X.shape[1]
    #     gradients = [np.zeros_like(w) for w in self.weights]
    #     delta = activations[-1] - y
    #     for i in range(m - 1, 0, -1):
    #         gradients[i] = np.dot(delta, activations[i - 1].T) / m
    #         delta = np.dot(self.weights[i].T, delta) * self.activation_derivative(weighted_inputs[i - 1])
    #     gradients[0] = np.dot(delta, X.T) / m
    #     return gradients
    
    # def update_parameters(self, gradients, learning_rate):
    #     for i in range(len(self.weights)):
    #         self.weights[i] -= learning_rate * gradients[i]
    #         self.biases[i] -= learning_rate * gradients[i]

    def update_parameters(self, gradients, learning_rate):
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * gradients['dw'][i]
            self.biases[i] -= learning_rate * gradients['db'][i]

    def train(self, X, y, epochs=1000, learning_rate=0.01, batch_size=4, patience=4):
        X = np.array(X).T  # Ensuring X is a NumPy array, transposed to match shape expectations
        y = np.array(y).T  # Ensuring y is transposed to match (num_classes, num_samples)
        
        # Setup for early stopping
        best_loss = float('inf')
        patience_counter = 0

        # Batch training implementation
        num_samples = X.shape[1]
        num_batches = num_samples // batch_size

        for epoch in range(epochs):
            for batch_index in range(num_batches):
                start_index = batch_index * batch_size
                end_index = start_index + batch_size
                X_batch = X[:, start_index:end_index]
                y_batch = y[:, start_index:end_index]

                activations, weighted_inputs = self.forward_propagation(X_batch)
                gradients = self.backward_propagation(X_batch, y_batch, activations, weighted_inputs)
                self.update_parameters(gradients, learning_rate)

            # Evaluate on validation set after each epoch
            if X_val is not None and y_val is not None:
                val_loss = self.evaluate(X_val, y_val)
                print(f"Epoch {epoch}, Validation Loss: {val_loss}")

                # Check for early stopping
                if val_loss < best_loss:
                    best_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1

                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    break

            # Optional: print training loss during training
            if epoch % 100 == 0 or epoch == epochs - 1:
                train_loss = self.evaluate(X, y)  # Evaluate on the entire training set
                print(f"Epoch {epoch}, Training Loss: {train_loss}")

        return

    
    def predict(self, X):
        activations, _ = self.forward_propagation(X)
        return activations[-1]
    
    def evaluate(self, X, y):
        X = np.array(X)
        y = np.array(y).T  # Ensure y is transposed
        predictions = self.predict(X)
        loss = cross_entropy(predictions, y)
        return loss
    
    # def train(self, X, y, epochs=1000, learning_rate=0.01):
    #     X = np.array(X) 
    #     y = np.array(y)
    #     for epoch in range(epochs):
    #         activations, weighted_inputs = self.forward_propagation(X)
    #         gradients = self.backward_propagation(X, y, activations, weighted_inputs)
    #         self.update_parameters(gradients, learning_rate)
    #         if epoch % 100 == 0:
    #             loss = cross_entropy(activations[-1], y)
    #             print(f"Epoch {epoch}, Loss: {loss}")

    # def predict(self, X):
    #     activations, _ = self.forward_propagation(X)
    #     return activations[-1]
    
    # def evaluate(self, X, y):
    #     predictions = self.predict(X)
    #     loss = cross_entropy(predictions, y)
    #     return loss
    
    def print_parameters(self):
        for i in range(len(self.weights)):
            print(f"Layer {i + 1}:")
            print("Weights:")
            print(self.weights[i])
            print("Biases:")
            print(self.biases[i])


# Define the target attribute
target_attribute = "Class"

# Split the data into training, validation, and testing sets using the neural_split_data function
X_train, y_train, X_val, y_val, X_test, y_test = neural_split_data(data, target_attribute)


# Define the architecture of the neural network
architecture = [14,10,8,6, 4]

num_classes = 4  # Adjust based on your specific dataset
y_train_encoded = one_hot_encode(y_train, num_classes)
y_val_encoded = one_hot_encode(y_val, num_classes)
y_test_encoded = one_hot_encode(y_test, num_classes)


# Instantiate the NeuralNetwork class
neural_network = NeuralNetwork(architecture, activation='relu')

# Train the neural network using the training data
neural_network.train(X_train, y_train_encoded, epochs=1000, learning_rate=0.01)

# Evaluate the neural network using the validation data
validation_loss = neural_network.evaluate(X_val, y_val_encoded)
print(f"Validation Loss: {validation_loss}")

# Print the parameters of the neural network
neural_network.print_parameters()

# Evaluate the neural network using the testing data
test_loss = neural_network.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {test_loss}")

# Predict the target attribute for the testing data
predictions = neural_network.predict(X_test)
print(predictions)

# # Evaluate the model using accuracy, precision, recall, and F1-score
# accuracy = accuracy_score(y_test, predictions)
# eval_accuracy = evaluation_metric.accuracy_score(y_test, predictions)

# precision = precision_score(y_test, predictions, average='weighted')
# eval_precision = evaluation_metric.precision_score(y_test, predictions, positive_label=1)

# recall = recall_score(y_test, predictions, average='weighted')
# eval_recall = evaluation_metric.recall_score(y_test, predictions, positive_label=1)

# f1 = f1_score(y_test, predictions, average='weighted')
# eval_f1 = evaluation_metric.f1_score(y_test, predictions, positive_label=1)

# Print the evaluation metrics
# print(f"Accuracy: {accuracy:.2f}")
# print(f"Eval Accuracy: {eval_accuracy:.2f}")

# print(f"Precision: {precision:.2f}")
# print(f"Eval Precision: {eval_precision:.2f}")

# print(f"Recall: {recall:.2f}")
# print(f"Eval Recall: {eval_recall:.2f}")

# print(f"F1-score: {f1:.2f}")
# print(f"Eval F1-score: {eval_f1:.2f}")  



Layer 1:
Activations shape: (14, 4)
Weights shape: (8, 10)


ValueError: shapes (8,10) and (14,4) not aligned: 10 (dim 1) != 14 (dim 0)